In [1]:
import pandas as pd
import numpy as np
import streamlit as st
import requests 
import matplotlib.pyplot as plt

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import FreqDist

2022-05-09 05:17:00.195 INFO    matplotlib.font_manager: generated new fontManager
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
#add title to page
st.title('Yelp Business Data')

#reading updated business and review csv
business = pd.read_csv('bussiness_updated.csv')
reviews = pd.read_csv('df_review_bus.csv')

#dropping some columns
business = business.drop(columns=['occurance_of_category','is_open'])

#displaying datframe of business data
st.dataframe(data=business, width=None, height=None)

#have user select category
with st.sidebar:
    st.subheader("User Inputs for Graph")
    category_chosen = st.sidebar.selectbox("Select a Category", reviews.categories.unique())

#subset of yelp data based on the category chosen 
category_subset = reviews[reviews['categories'] == category_chosen]

#taking all reviews from dataframe, joining them into one string, and making all the
#characters lowercase 
reviews_txt = ' '.join(category_subset['text']).lower()

#splitting into tokens
word_tokens = word_tokenize(reviews_txt)

#making list of stop words
stop_words = set(stopwords.words('english'))

#filterng out stop words and none words
words = []
for word in word_tokens:
    if word.isalpha() and word not in stop_words:
        words.append(word)

#getting frequency distrabution
frequency = FreqDist(words)

#creating dataframes
dist = pd.DataFrame(frequency.items(),columns=['word', 'freq'])

#sorting by frequency 
dist = dist.sort_values(by='freq', ascending=False).reset_index(drop=True)

#selecting only top 10
dist = dist.iloc[:10, :]

#getting data for graph 
words = dist['word']
frequency = dist['freq']

#create figure
fig = plt.figure()
#title for graph 
title = f"10 Most Frequent Words in Yelp Reviews for {category_chosen} Businesses"
#setting up graph and labels
plt.barh(words, frequency)
plt.ylabel('Word')
plt.xlabel('Word Frequency')
plt.title(title)
st.pyplot(fig)

st.write("Only includes reviews for top 1% of businesses with the most reviews within each category")



,word,freq
0,great,73
1,place,63
2,time,62
3,one,57
4,store,53
5,love,48
6,get,47
7,experience,45
8,service,42
9,always,36


In [ ]:
#reading business csv
business = pd.read_csv('business.csv')

#splitting the string of categories for each row into a list 
categories = business['categories'].str.split(", ")

#replacing the categories column with a list for each row 
business['categories'] = categories

#creating a list of all the categories by iterating through the nested list 
#of categories and flattening the list 
flatlist = []
for sublist in categories:
    for element in sublist:
        flatlist.append(element)

#removing duplicates from the flattened list of categories 
#category_list = []
#for category in flatlist:
 #   if category not in category_list:
  #     category_list.append(category)

#changing flattened list of all categories into pandas series and counting occurance 
#of each category then creating a dictionary
categories_series = pd.Series(flatlist)
occurances = categories_series.value_counts()
occurances_dict = occurances.to_dict()

# keeping only one category for each business based on which category has the 
# most businesses

#add column of number of occurances
business_exploded = business.explode('categories')
occur = business_exploded['categories'].map(occurances_dict)
business_exploded['occurance_of_category'] = occur

#sort rows by business id then occurance of category so the same business is kept 
#together and the most popular category for the business will be listed first
business = business_exploded.sort_values(by=['business_id','occurance_of_category'], ascending=False)

#keeping only one row for each business that has the most popular category
business = business.drop_duplicates(subset='business_id', keep='first')

# keeping only businesses that in top 1 percentile of businesses with most of reviews within each category 
rank = business.groupby('categories')['review_count'].rank(pct=True)
business['percent_rank'] = rank
top_reviewed_bus = business[business['percent_rank'] >= 0.99]

bus_id_list = top_reviewed_bus['business_id'].tolist()

review_df = pd.DataFrame()

#load csv file i just created
df_yelp_data = pd.read_csv('df_review_bus.csv')

#change time_created to date
df_yelp_data['time_created']= pd.to_datetime(df_yelp_data['time_created'])

#go back and change for streamlit

#category_chosen = st.sidebar.selectbox("Select a Category", df_yelp_data.categories.unique())
####category_chosen = 'Restaurants'

#subset of yelp data based on the ctegory chosen 
####category_subset = df_yelp_data[df_yelp_data['categories'] == category_chosen]

#filtering catergory subset for only reviews after or on march 1 2022
####aft_mar_2022 = category_subset[category_subset['time_created'] >= 'March 01, 2022 00:00']
aft_mar_2022 = df_yelp_data[df_yelp_data['time_created'] >= 'March 01, 2022 00:00']

#filtering caeroy subset to get only reviews from during major covid period between 
#march 2020 and march 2022
bef_mar_2022 = df_yelp_data[df_yelp_data['time_created'] < 'March 01, 2022 00:00']
covid = bef_mar_2022[bef_mar_2022['time_created'] > 'March 01, 2020 00:00']


#taking all of the reviews and joininging them into one long string and making all 
#of the letters lowercase 
txt_aft_2022 = ' '.join(aft_mar_2022['text']).lower()
txt_covid = ' '.join(covid['text']).lower()

stop_words = set(stopwords.words('english'))

# iterating over list of words used during covid 
#using if statment to filter out anything that isnt a word and stop words
#is_alpha method returns True if all the characters in the string are letters (a-z).
covid_words = []
for word in word_tokens_covid:
    if word.isalpha() and word not in stop_words:
        covid_words.append(word)

#amount_of_covid_words = len(covid_words)

#doing same thing for words from after covid
after_words = []
for word in word_tokens_after:
    if word.isalpha() and word not in stop_words:
        after_words.append(word)

#amount_of_words_a = len(after_words)

#using The FreqDist function from nltk library to get frequency distribution 
#of all the words in each list 
covid_frequency = FreqDist(covid_words)
after_frequency = FreqDist(after_words)

#creating a dataframe for each time period 
covid_dist = pd.DataFrame(covid_frequency.items(),columns=['word', 'freq_covid'])
after_dist = pd.DataFrame(after_frequency.items(),columns=['word', 'freq_after'])
#with the word 20 most used words and their frequency 
#covid_dist = pd.DataFrame(covid_frequency.most_common(20),columns=['word', 'freq_covid'])
#after_dist = pd.DataFrame(after_frequency.most_common(20),columns=['word', 'freq_after'])

#making a list of the frquency of each word used doing covid 
covid_frequency = covid_dist["freq_covid"].tolist()

#getting sum of all frequencies 
total_frequencies = sum(covid_frequency)

#iteratting through the frequency of covid words and dividing it by the sum of 
#all frquencies to calculate percentage and appending it to a list 
covid_percentage = []
for freq in covid_frequency:
    percent = freq/total_frequencies
    #formated = "{:.2%}".format(percent)
    covid_percentage.append(percent)

#adding percentage to the dataframe
covid_dist['percent_covid_freq'] = covid_percentage

covid_dist

#doing the same thing for after covid words 

#making a list of the frquency of each word used doing covid
a_frequency = after_dist["freq_after"].tolist()

#getting sum of all frequencies 
total_frequencies = sum(covid_frequency)

#iteratting through the frequency of words after covid and dividing it by the sum of 
#all frquencies to calculate percentage and appending it to a list 
a_percentage = []
for freq in a_frequency:
    percent = freq/total_frequencies
    a_percentage.append(percent)

#adding percentage to the dataframe
after_dist['percent_after_freq'] = a_percentage

after_dist

#merging the dataframes for both time periods 
df_freq_words = pd.merge(covid_dist, after_dist, on='word',how='outer')

#sort words were in both tables but some were only in one so there are 
#a lot of nas so filling them in with zeros 
df_freq_words.fillna(0)

# if positive then mentioned more during covid 
df_freq_words['percent_diff'] = df_freq_words['percent_covid_freq'] - df_freq_words['percent_after_freq']




##### Title and intro

#st.title( 'Example Streamlit App' )
#st.write( '''
#This app is very small and does almost nothing.
#It's just an example.
#''' )


##### Inputs

#st.header( 'Choose two numbers' )
#a = st.slider( label='a', min_value=1, max_value=10, value=2, step=1 )
#b = st.slider( label='b', min_value=1, max_value=10, value=3, step=1 )


##### Output

#st.header( 'Tiny computations')
#st.write( f'{a} + {b} = {a+b}' )
#st.write( f'{a} - {b} = {a-b}' )
#st.write( f'{a} * {b} = {a*b}' )
#st.write( f'{a} / {b} = {a/b}' )

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3c5109e0-d4f5-47cd-9c2e-1a9d1a674c6f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>